# Create Zephir Dataset for US Fed Doc analysis

**Zephir Notebook**
**This notebook can only be run with database credentials. It is meant for internal use by the Zephir team, and published to document the process.**

The following dataset was built for overlap analysis between Zephir and the HathiTrust US Federal Documents Registry. The reasons for the analysis are detailed in the [analysis notebook](htusfd_suggestion_analysis).

This dataset lists all HathiTrust volumes from the Zephir database that: 
  1. Have been ingested into the repository.
  2. Have one or more oclc numbers. 
  
Columns
* **zdb_row** (zephir_records.autoid): Internal row in the Zephir database. 1-1 relationship with HTID.
* **oclc** (zephir_identifiers.identifier/type=oclc): OCLC number associated with a HTID.
* **cid** (zephir_records.cid): Cluster identifier associatied with the HTID. 1-N relationship with HTID.
* **contributor_code** (zephir_records.source): The code associated with a contributor.
* **collection_code** (zephir_records.source_collection): The code associated with a collection. 1-N relationship with contributor code, but often a repeat of the contributor_code.
* **contribsys_id** (zephir_records.contribsys_id): The bibiliographic record identifier in a contributors metadata system (e.g. ILS). 1-N relationship with HTIDs for multiple holdings.
* **htid** (zephir_records.id): Hathitrust unique identifier for a volume.
* **usfd_status** (zephir_records.var_usfeddoc): A boolean value indicating if the associated metadata 008 field contains designations for a federal document printed in the US.

Notes
* There is a row per OCLC-volume match. This means there may be multiple rows for the same volume if the volume has multiple OCLC.
* The dataset is ordered by by the zephir_records autoid field for optimization.
* Leading zeros were trimmed from OCLC numbers to match normalization in the US Fed Docs registry
* The 008 desiginations are a proxy for US Federal Documents, as sometimes the US government publshes documents outside of the US and other countries may puplish documents in the US.
* Field names were exported as the first row header. This is important for splitting and importing into pandas later.

### Create export from database

#### Imports

In [ ]:
import datetime
import collections
import getpass
import json
import os



import pandas
import pymysql

#### Define SQL Query

In [ ]:
select_stmt = "select zr.autoid as zdb_row, trim(leading '0' from zi.identifier) as oclc, zr.cid, source as contributor_code, source_collection as collection_code, contribsys_id, zr.id as htid, zr.var_usfeddoc as usfd_status from zephir_identifiers as zi inner join zephir_identifier_records as zir on zi.autoid = zir.identifier_autoid inner join zephir_records as zr on zir.record_autoid = zr.autoid where zi.type = 'oclc' and zr.attr_ingest_date is not null order by zr.autoid ASC"

#### Get Connection & Credentials
Enter database connection information. Everything is treated like a password to avoid accidental leaking into jupyter notebook.

##### Enter Host

In [ ]:
host = getpass.getpass()

##### Enter Username

In [ ]:
user = getpass.getpass()

##### Enter Password

In [ ]:
password = getpass.getpass()

In [ ]:
# create connection
conn = pymysql.connect(
    host=host,
    port=int(3306),
    user=user,
    passwd=password,
    db="htmm")

#### Running Querying 
This is going to take awhile.

In [ ]:
query_datetime = datetime.datetime.utcnow().isoformat()
df = pandas.read_sql_query(select_stmt,
    conn)
df = df.sort_values(by=['zdb_row'])

#### Data munge to the right column/datatype

In [ ]:
# cast usfd as bool
df['usfd_status'] = df['usfd_status'].astype('bool')

#### Write datset to disk as csv file
This will also take awhile.

In [ ]:
dataset_name = "zephir-oclc-feddocs-dataset"
dataset_file = "data/{}.csv".format(dataset_name)

# csv settings
header = True
encoding = "utf8"
delimiter = ","

if not os.path.exists("data"):
    os.makedirs("data")
    
# write to disk
df.to_csv(dataset_file, index=False, header=header, encoding=encoding, sep=delimiter)

#### Write manfifest to disk

In [ ]:
manifest = {}
manifest["name"] = dataset_name
manifest["description"] = "Dataset of all Zephir OCLC entries and their USFD status. Duplicates in cases of volummes with more than one OCLC."
manifest["datetime"] = str(datetime.datetime.utcnow().isoformat())

# derive schema from dataframe
schema = collections.OrderedDict()
for column in df.columns.array:
    schema[str(column)]= str(df.dtypes[column])
manifest["schema"] = schema

# note format to manifest
manifest["format"] = {
 "type": "delimited",
 "extension": "csv",
 "delimiter": delimiter,
 "header": header,
 "encoding": encoding,
 }

manifest["data-origins"] = [{
    "origin": "HTMM production database (Zephir)",
    "datetime": str(query_datetime)
}]
manifest_file = "data/{}.manifest.json".format(manifest["name"])
with open(manifest_file, 'w') as outfile:
    json.dump(manifest, outfile, indent=4)

#### Finishing up!

In [ ]:
print("Completed notebook ({}).".format(datetime.datetime.utcnow().isoformat()))
print("Output created:")
print(dataset_file)
print(manifest_file)